In [ ]:
import dash
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table  
from dash.dependencies import Input, Output
import plotly.graph_objs as go 
import plotly.figure_factory as ffimport 
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import itertools
import ccxt
import ccxt.websocket_support
import json

# ccxt.binance = ccxtv.binance
#ccxt.bitstamp = ccxt.websocket_support.bitstamp
#ccxt.bitmex = ccxt.websocket_support.bitmex
#ccxt.coinbasepro = ccxt.websocket_support.coinbasepro

# Define my world
spot_exchanges_2 = ['bitstamp','coinbasepro','kraken','liquid','bitfinex','gemini','binance',
                'bittrex','bitbank','huobipro','poloniex','bithumb']
spot_exchanges = ['bitstamp','coinbasepro']
exch_dict={}
for x in spot_exchanges:
    exec('exch_dict[x]=ccxt.{}()'.format(x))

Xpto= ['BTC','ETH','XRP','XMR','BCH','EOS','USDT','USDC','TRX','XLM','BSV','XBT','CSP']
Fiat=['USD','EUR','GBP','CHF','HKD','JPY','CNH','KRW']
xpto_fiat = [xpto+'/'+ fiat for xpto in Xpto for fiat in Fiat]
xpto_xpto = [p[0]+'/'+p[1] for p in itertools.permutations(Xpto,2)]

all_pairs = set(sum(itertools.chain([*exch_dict[x].load_markets()] for x in exch_dict),[])) 
pairs = list(set(xpto_fiat + xpto_xpto) & set(all_pairs))
pairs.sort()

def get_exchanges_for_pair(pair):
    '''input: a pair
    output: a dictionary of ccxt exchange objects of the exchanges listing the pair
    '''
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}

def get_order_books(pair):
    '''pair is the pair string ,'BTC/USD'...
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    ex = get_exchanges_for_pair(pair)
    nobinance= {key:value for key, value in ex.items() if key != 'binance'and  key != 'bitfinex'}
    order_books = {key: value.fetch_order_book(pair,limit=2000 if key!='bithumb' else 50,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
    if 'binance' in ex:
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    if 'bitfinex' in ex:
        order_books['bitfinex'] =  ex['bitfinex'].fetch_order_book(pair,limit=2000)
    return order_books

def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a dict of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    bids = []
    for x in dict_of_order_books:
        for bid in dict_of_order_books[x]['bids']:
            bids.append(bid+[x])
    asks = []
    for x in dict_of_order_books:
        for ask in dict_of_order_books[x]['asks']:
            asks.append(ask+[x])
    agg_dict_order_book['bids'] = (pd.DataFrame(bids)).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(asks)).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size','exc'])
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size','exc'])
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size_$'] = bid_side['bid_size']*bid_side['bid']
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side['ask_size_$'] = ask_side['ask_size']*ask_side['ask']
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill'] = normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bids_exc']=bid_side.groupby('bid%',sort=False).apply(lambda x: x['exc'].loc[x['bid_size'].idxmax()])
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['asks_exc']=ask_side.groupby('ask%',sort=False).apply(lambda x: x['exc'].loc[x['ask_size'].idxmax()])
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book

def build_book(order_books,pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

def plot_book(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    ''' plots the order book as a v shape chart '''
    order_book = build_book(order_books,pair,exc,cutoff)
    best_bid = round(order_book['bid'].max(),3)
    best_ask = round(order_book['ask'].min(),3)
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    if relative:
        figure = go.Figure(data=[{'x':order_book.index,'y':order_book['cum_ask_size'+col_to_chart],'name':'asks'}
                      ,{'x':order_book.index,'y':order_book['cum_bid_size'+col_to_chart],'name':'bids'}],
     layout={'title':' / '.join(exc), 'xaxis':{'title':pair +'  ' + str(best_bid)+' / '+ str(best_ask)}})
    else:
        figure= go.Figure(data=[{'x':order_book['ask'].fillna(0)+order_book['bid'].fillna(0),'y':order_book['cum_ask_size'+col_to_chart],'name':'asks'}
                      ,{'x':order_book['ask'].fillna(0)+order_book['bid'].fillna(0),'y':order_book['cum_bid_size'+col_to_chart],'name':'bids'}],
     layout={'title':' / '.join(exc), 'xaxis':{'title':pair +'  ' + str(best_bid)+' / '+ str(best_ask)}})
    return figure

def plot_depth(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    order_book = build_book(order_books,pair,exc,cutoff)
    mid = (order_book['bid'].max()+order_book['ask'].min())/2 if relative else 1
    figure = go.Figure(data=[{'x':order_book['cum_ask_size'+col_to_chart],'y':order_book['average_ask_fill']/mid,'name':'ask depth'},
                            {'x':- order_book['cum_bid_size'+col_to_chart],'y':order_book['average_bid_fill']/mid,'name':'bid depth'}],
    layout={'title': 'Market Depth'})

    return figure

def order_fill(order_book_df, order_sizes,in_ccy=True):
    '''takes in an order book dataframe and an np.array of order sizes
        with size in currecncy by default else in coin
        returns an np.array of the purchase costs or the sale proceeds of an order
    '''
    average_fills = np.zeros(order_sizes.shape)
    mid=(order_book_df['ask'].min()+order_book_df['bid'].max())/2
    print(mid)
    if in_ccy:
        order_sizes=order_sizes/mid
    for i , order_size in enumerate(order_sizes):
        if order_size > 0:
            last_line = order_book_df[order_book_df['cum_ask_size']>order_size].iloc[0]
            ccy_fill = last_line['cum_ask_size_$']+(order_size-last_line['cum_ask_size'])*last_line['ask']
            average_fill=ccy_fill/order_size
        elif order_size < 0:
            last_line=order_book_df[order_book_df['cum_bid_size'] > -order_size].iloc[0]
            ccy_fill=last_line['cum_bid_size_$']+(-order_size-last_line['cum_bid_size'])*last_line['bid']
            average_fill = -ccy_fill/order_size
        average_fills[i]=average_fill
    return average_fills


In [ ]:
pair = 'BTC/USD'
#pair='CSP/BTC'

In [ ]:
%%timeit
order_books=get_order_books(pair)

In [ ]:
order_books

In [ ]:
aggregated = aggregate_order_books(order_books)

In [ ]:
normalized=normalize_order_book(aggregated)

In [ ]:
normalized

In [ ]:
order_books.keys()

In [ ]:
exc=get_exchanges_for_pair(pair)
exc

In [ ]:
from plotly.offline import plot, iplot , init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
dplot=plot_book(order_books,pair,exc)

In [ ]:
dplot2=plot_depth(order_books,pair,exc)

In [ ]:
iplot(dplot)

In [ ]:
iplot(dplot2)

In [ ]:
def get_pairs_for_exchange(ex):
    '''input: an exchange
    output: a list of pairs '''
    d={}
    exec('d[ex]=ccxt.{}()'.format(ex))
    d[ex].load_markets()
    return d[ex].symbols

In [ ]:
[ p for p in get_pairs_for_exchange('kraken') if 'GBP' in p]

In [ ]:
fees={x:ex.fees for x,ex in exch_dict.items()}

In [ ]:
fees.keys()

In [ ]:
feest=pd.DataFrame(fees).T

In [ ]:
feest['taker']=feest['trading'].apply(lambda x:x['taker'] if 'taker' in x.keys() else np.nan)

In [ ]:
feest['funding']['coinbasepro']

In [ ]:
feest['deposit']=feest['funding'].apply(lambda x:x['deposit'])

In [ ]:
feest['withdrawal']=feest['funding'].apply(lambda x:x['withdraw'])

In [ ]:
feest.drop(['funding','trading'],axis=1).iloc[0]['deposit']

In [ ]:
feest.drop(['funding','trading'],axis=1).iloc[0]['withdrawal']

In [ ]:
feest.loc['kraken']['withdrawal']

In [ ]:
order_books.keys()

In [ ]:
import json
import datetime as dt

In [ ]:
save_this=json.dumps((order_books,dt.datetime.now().strftime("%Y-%m-%d  %H:%M:%S")))

In [ ]:
time_snap=json.loads(save_this)[1]

In [ ]:
(dt.datetime.now()-dt.datetime.strptime(time_snap,"%Y-%m-%d  %H:%M:%S")).seconds

In [ ]:
normalized

In [ ]:
def get_liq_params(normalized):

    best_bid = normalized['bid'].max()
    best_ask = normalized['ask'].min()
    mid= (best_bid+best_ask)/2
    spread = best_ask-best_bid
    spread_pct = spread/mid
    cross = min(0,spread)
    cross_pct = min(0,spread_pct)
    return pd.DataFrame([best_bid,best_ask,mid,spread,spread_pct,cross,cross_pct],index=['bid','ask','mid','spread','spread%','cross','cross%']).T

In [ ]:
get_liq_params(normalized)

In [ ]:
coinmarketcap=ccxt.coinmarketcap()

In [ ]:
coinmarketcap.load_markets()

In [ ]:
coinmarketcap.symbols

In [ ]:
order_books['bitstamp']

In [ ]:
kucoin=ccxt.kucoin2()